<a href="https://colab.research.google.com/github/anityu45/Indic-auto-complete-engine/blob/main/Deutsch%E2%80%93Jozsa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

!pip install qiskit qiskit-aer ipywidgets matplotlib pylatexenc --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 89.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 98.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 73.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 60.4 MB/s eta 0:00:00


In [2]:
import logging
logging.getLogger('stevedore.extension').setLevel(logging.ERROR)


In [3]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer.backends import AerSimulator
from qiskit.visualization import plot_histogram
from IPython.display import display, clear_output
import ipywidgets as widgets


In [4]:
def constant_oracle(n_qubits, value=0):
    qc = QuantumCircuit(n_qubits + 1, name=f"const_{value}")
    if value == 1:
        qc.x(n_qubits)
    return qc

# ---- Balanced oracle from truth table ----
def balanced_oracle_from_map(bitmap):
    n_qubits = max(1, (max(bitmap.keys()).bit_length()))
    qc = QuantumCircuit(n_qubits + 1, name="balanced")
    for x, fx in bitmap.items():
        if fx == 1:
            bits = [(x >> i) & 1 for i in range(n_qubits)]
            for i, bit in enumerate(bits):
                if bit == 0:
                    qc.x(i)
            qc.mcx(list(range(n_qubits)), n_qubits)
            for i, bit in enumerate(bits):
                if bit == 0:
                    qc.x(i)
    return qc

# ---- Deutsch–Jozsa circuit builder ----
def deutsch_jozsa_circuit(n, oracle):
    qc = QuantumCircuit(n + 1, n)
    qc.x(n)
    qc.h(range(n + 1))
    qc.append(oracle, range(n + 1))
    qc.h(range(n))
    qc.measure(range(n), range(n))
    return qc

# ---- Interactive widgets ----
n_qubits_slider = widgets.IntSlider(min=1, max=4, step=1, description='Number of qubits:')
oracle_type_radio = widgets.RadioButtons(
    options=['Constant (f(x)=0)', 'Constant (f(x)=1)', 'Balanced (example)'],
    description='Oracle type:')
run_button = widgets.Button(description="Run Deutsch-Jozsa", button_style='success')
output_area = widgets.Output()

display(n_qubits_slider, oracle_type_radio, run_button, output_area)

# ---- Button click event ----
def on_button_click(b):
    with output_area:
        clear_output()
        n = n_qubits_slider.value
        oracle_type = oracle_type_radio.value

        # --- Choose oracle ---
        if oracle_type == 'Constant (f(x)=0)':
            oracle = constant_oracle(n, 0)
        elif oracle_type == 'Constant (f(x)=1)':
            oracle = constant_oracle(n, 1)
        else:  # Balanced examples
            if n == 1:
                bitmap = {0: 0, 1: 1}
            elif n == 2:
                bitmap = {0: 0, 1: 1, 2: 0, 3: 1}
            elif n == 3:
                bitmap = {i: (i % 2) for i in range(8)}
            elif n == 4:
                bitmap = {i: bin(i).count('1') % 2 for i in range(16)}
            else:
                print(f"No balanced oracle defined for {n} qubits.")
                return
            oracle = balanced_oracle_from_map(bitmap)

        # --- Build & simulate circuit ---
        dj_circuit = deutsch_jozsa_circuit(n, oracle)
        simulator = AerSimulator()
        job = simulator.run(transpile(dj_circuit, simulator), shots=1024)
        result = job.result()
        counts = result.get_counts()

        # --- Display results ---
        print("Deutsch–Jozsa Circuit:")
        display(dj_circuit.draw(output='mpl'))
        print("\nMeasurement Results:")
        print(counts)
        display(plot_histogram(counts))

        if '0'*n in counts and len(counts) == 1:
            print("\n✅ Result: Function is CONSTANT")
        else:
            print("\n⚖️ Result: Function is BALANCED")

run_button.on_click(on_button_click)

IntSlider(value=1, description='Number of qubits:', max=4, min=1)

RadioButtons(description='Oracle type:', options=('Constant (f(x)=0)', 'Constant (f(x)=1)', 'Balanced (example…

Button(button_style='success', description='Run Deutsch-Jozsa', style=ButtonStyle())

Output()